In [1]:
import wikipedia
import os
import json
import tqdm
import argparse

import multiqa_utils.general_utils as gu
import multiqa_utils.qampari_utils as qu
import multiqa_utils.retrieval_utils as ru
import multiqa_utils.wikipedia_utils as wu
import multiqa_utils.distributed_utils as du

%load_ext autoreload
%autoreload 2

## Setup Everything

In [2]:
path_args = gu.current_default_path_args()

In [8]:
text_titles = wu.build_gt_wikititle_set(path_args, force=False)

>> Index already exists: /scratch/ddr8143/wikipedia/tagme_dumps_qampari_wikipedia/postprocessed/gt_title_set.json


In [10]:
notext_titles = json.load(open(path_args.no_text_pages))

In [9]:
curr_cache = wu.get_initial_str2wikipage_cache(text_titles, path_args, force=False)

>> Loading the cache
>> Initial cache size: 7198496


In [13]:
qmp_dev = qu.load_dev_data()

In [11]:
gpt_ans = json.load(open(path_args.gpt_ans_path))

In [12]:
elq_ans = gu.loadjsonl(path_args.elq_ans_path)

## Try Retrieval

In [ ]:
# TODO: this!

In [17]:
qmp_ex = qmp_dev[0]
qmp_ex['question_text']

'What manga was drawn by Ryoichi Ikegami?'